
<h1>Preprocessing Task</h1>



In [ ]:
#import excel file
#Path: C:\Users\user\Downloads\ISE_CoffeeTransaction_April2023.xlsx
import pandas as pd
from datetime import datetime

In [ ]:
#Open all sheets
Transaction = pd.read_excel(r'C:\Users\user\Downloads\ISE_CoffeeTransaction_April2023.xlsx', sheet_name='Transactions')
Account = pd.read_excel(r'C:\Users\user\Downloads\ISE_CoffeeTransaction_April2023.xlsx', sheet_name='Account')
Location = pd.read_excel(r'C:\Users\user\Downloads\ISE_CoffeeTransaction_April2023.xlsx', sheet_name='location')
Product = pd.read_excel(r'C:\Users\user\Downloads\ISE_CoffeeTransaction_April2023.xlsx', sheet_name='Product')

In [ ]:
#combine the four sheets into one sheet named df use Transaction as the main sheet which combine with Account using Account_id, Location using Location, Product using product_id
df = pd.merge(Transaction, Account, on='Account_id')
df = pd.merge(df, Product, on='product_id')

In [ ]:
#sort df by Account_id, transaction_date, time_to_purchase
df = df.sort_values(by=['Account_id', 'transaction_date', 'time_to_purchase'])

In [ ]:
#drop the columns which are not needed
df = df.drop(['product_id', 'Location'], axis=1)
df = df.drop(['Unnamed: 3'], axis=1)

In [ ]:
#Change the nan value in column gas_type to no gas
df['gas_type'] = df['gas_type'].fillna('no gas')
#change registration_date column to datetime format and column name to membership_start_date
df['transaction_date'] = pd.to_datetime(df['transaction_date'])
df = df.rename(columns={'Register Date': 'membership_start_date'})
#Change the nan value in column membership_start_date to no membership
df['membership_start_date'] = df['membership_start_date'].fillna('no membership')

In [ ]:
#add another column which is time of the day like morning, afternoon, evening, night
df['time_of_the_day'] = df['time_to_purchase'].apply(lambda x: 'morning' if x < datetime.time(datetime.strptime('12:00:00', '%H:%M:%S')) else ('afternoon' if datetime.time(datetime.strptime('12:00:00', '%H:%M:%S')) <= x < datetime.time(datetime.strptime('18:00:00', '%H:%M:%S')) else ('evening' if datetime.time(datetime.strptime('18:00:00', '%H:%M:%S')) <= x < datetime.time(datetime.strptime('23:59:59', '%H:%M:%S')) else 'night')))


<h1>First Model (Correlation analysis)</h1>

In [ ]:
#Recieve input from user as time_of_day
time_of_day = input("Enter the time of the day (morning, afternoon, evening, all day): ")
#recieve input from user as product_recommend
product_recommend = input("Enter the product name(Cappuccino, Tea, RedBull, Bagel, Espresso, Muffin, Latte, Hot Chocolate, Croissant, Coffee): ")

Enter the time of the day (morning, afternoon, evening, all day): morning
Enter the product name(Cappuccino, Tea, RedBull, Bagel, Espresso, Muffin, Latte, Hot Chocolate, Croissant, Coffee): Tea


In [ ]:
#create a user-item matrix
if time_of_day == 'all day':
    user_item_matrix = pd.pivot_table(df, index='Account_id', columns='product_name', values='transaction_amount', aggfunc='count', fill_value=0)
else:
    user_item_matrix = pd.pivot_table(df[df['time_of_the_day'] == time_of_day], index='Account_id', columns='product_name', values='transaction_amount', aggfunc='count', fill_value=0)

In [ ]:
#calculate the correlation matrix based on the user-item matrix
item_correlation = user_item_matrix.corr(method='pearson')
#Choose the product with the most sales
target_product = product_recommend
#Get the correlation values of the target product with other products
target_product_correlations = item_correlation[target_product]

In [ ]:
#sort the correlations in descending order
recommended_products = target_product_correlations.sort_values(ascending=False)
#filter out the target product itself
recommended_products = recommended_products.drop(target_product)

In [ ]:
#Print the Best sales
if time_of_day == 'all day':
    Best_Sales = df.groupby('product_name')['product_price'].sum().sort_values(ascending=False)
else:
    Best_Sales = df[df['time_of_the_day'] == time_of_day].groupby('product_name')['product_price'].sum().sort_values(ascending=False)
print(f"Best Sales at {time_of_day} ")
print(Best_Sales.head().to_string(header=False, float_format=lambda x: '฿{:.2f}'.format(x)))

Best Sales at morning 
RedBull      ฿28060.00
Cappuccino    ฿4683.00
Latte         ฿4595.50
Muffin        ฿4550.00
Croissant     ฿4200.00


In [ ]:
#Print the recommended products
print(f"Recommended Product for {product_recommend} at {time_of_day}")
print(recommended_products.head().to_string(header=False, float_format=lambda x: '{:.2%}'.format(x)))

Recommended Product for Tea at morning
Latte       25.95%
Bagel       25.76%
Espresso    23.57%
Croissant   23.16%
Coffee      22.91%


<h1>Second Model (GradientBoostingClassifier)</h1>

In [ ]:
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

In [ ]:
#filter df to only the column that have membership
df = df[df['membership_start_date'] != 'no membership']
#add month column by using the month of membership_start_date
df['membership_start_date'] = pd.to_datetime(df['membership_start_date'])
df['month'] = df['membership_start_date'].dt.month

In [ ]:
#Convert transaction_date column to numerical features
df['transaction_date'] = pd.to_datetime(df['transaction_date'])
df['year'] = df['transaction_date'].dt.year
df['month'] = df['transaction_date'].dt.month
df['day'] = df['transaction_date'].dt.day
df['hour'] = df['transaction_date'].dt.hour
df['minute'] = df['transaction_date'].dt.minute
df['second'] = df['transaction_date'].dt.second

In [ ]:
#Drop original transaction_date and time_to_purchase columns
df = df.drop(['transaction_date', 'time_to_purchase','membership_start_date','transaction_id'], axis=1)
#convert categorical variables into numerical representations
df = pd.get_dummies(df, columns=['gas_type', 'time_of_the_day'], drop_first=True)

In [ ]:
#split the data into training and testing sets
X = df.drop('product_name', axis=1)
y = df['product_name']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)

In [ ]:
#Train the model
model = GradientBoostingClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
#Evaluate the model
print(classification_report(y_test, predictions))

               precision    recall  f1-score   support

        Bagel       0.09      0.06      0.07       209
   Cappuccino       0.10      0.13      0.11       224
       Coffee       0.11      0.05      0.07       242
    Croissant       0.11      0.09      0.10       254
     Espresso       0.12      0.12      0.12       219
Hot Chocolate       0.12      0.11      0.12       115
        Latte       0.11      0.09      0.10       252
       Muffin       0.15      0.07      0.10       240
      RedBull       0.68      1.00      0.81      1010
          Tea       0.08      0.05      0.06       241

     accuracy                           0.39      3006
    macro avg       0.17      0.18      0.16      3006
 weighted avg       0.30      0.39      0.33      3006

